# binary url encoding
Kunnen we codes handig van string naar bitmap omzetten en terug?

Yessur https://docs.python.org/2/library/binascii.html#binascii.hexlify

In [ ]:
import pickle
import binascii

In [ ]:
test_bools = [True, False, False, True]

In [ ]:
bytearray(test_bools)

In [ ]:
binascii.hexlify(bytearray(test_bools))

In [ ]:
list(binascii.unhexlify(binascii.hexlify(bytearray(test_bools))))

Hm, nee dat is niet echt de compacte representatie die ik in gedachten had... oh dit is beter: https://docs.python.org/3/library/base64.html

In [ ]:
bin([True, False])

In [ ]:
def bool_list_to_binary_int(bool_list):
    return sum(int(v)*2**i for i,v in enumerate(bool_list[::-1]))

In [ ]:
def bool_list_to_binary_repr(bool_list):
    return bin(bool_list_to_binary_int(bool_list))

In [ ]:
bool_list_to_binary_repr(test_bools).encode('base64')

In [ ]:
str(bool_list_to_binary_int(test_bools)).encode('base64')

In [ ]:
bytes(bool_list_to_binary_repr(test_bools).encode('ascii'))

In [ ]:
import base64

In [ ]:
base64.urlsafe_b64encode(str(bool_list_to_binary_int(test_bools)))

In [ ]:
import BitVector

In [ ]:
bv = BitVector.BitVector(bitlist=test_bools)

In [ ]:
print(bv)

In [ ]:
import io
import codecs

tests met io weer weggehaald

In [ ]:
bv = BitVector.BitVector(bitlist=test_bools * 2)

In [ ]:
print(bv)

In [ ]:
with open('testhy.txt', 'wb') as fh:
    bv.write_to_file(fh)

In [ ]:
with open('testhy.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [ ]:
brap, proep

In [ ]:
bv_lang = BitVector.BitVector(bitlist=test_bools * 8)

In [ ]:
print(bv_lang)

In [ ]:
with open('testlang.txt', 'wb') as fh:
    bv_lang.write_to_file(fh)

In [ ]:
with open('testlang.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [ ]:
brap, proep

In [ ]:
bv_langer = BitVector.BitVector(bitlist=test_bools * 16)

In [ ]:
with open('testlanger.txt', 'wb') as fh:
    bv_langer.write_to_file(fh)

In [ ]:
with open('testlanger.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')

In [ ]:
brap, proep

In [ ]:
bv_langst = BitVector.BitVector(bitlist=test_bools * 32)

In [ ]:
with open('testlangst.txt', 'wb') as fh:
    bv_langst.write_to_file(fh)
with open('testlangst.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')
brap, proep

In [ ]:
bv_lung = BitVector.BitVector(bitlist=[1, 1, 1, 0] * 64 * 3)
with open('testlung.txt', 'wb') as fh:
    bv_lung.write_to_file(fh)
with open('testlung.txt', 'rb') as fh:
    brap = fh.read()
    proep = codecs.encode(brap, 'base64')
brap, proep

Die ='s aan het eind komen omdat base64 3x8=24 bits in 4 karakter opslaat, dus 6 bits per karakter. Als je dus geen factor 6 erin hebt dan krijg je een = aan het eind, wrs iets van de lege extra karakter ofzo.

Idd, zie hier, het is padding: https://stackoverflow.com/a/36571117/1199693

Kun je in een URL dus wel weglaten, alleen moet je ze dan weer toevoegen als je encoding string geen multiple van 4 is.

In [ ]:
codecs.encode([True, False], 'base64')

In [ ]:
byte_file_like = io.BytesIO()

In [ ]:
dinkie = [True, False, False, True, True, False, False, True]

In [ ]:
byte_file_like.write(bytes(codecs.encode(str(dinkie), encoding='utf-8')))

In [ ]:
byte_file_like.seek(0)

In [ ]:
byte_file_like.read()